# Audio signal presenter

## 0. Setup

### 0.1 Load Packages

In [23]:
import pygame
import time
import os
import random
from collections import Counter
import serial

### 0.2 Set random seed to ensure reproducibility

In [24]:
random.seed(42)

### 0.3 Constants

In [25]:
NUM_REPS = 3               
BLOCK_LENGTH = 1            

TAGS = ['dog',
        'environment',
        'human',
        'swine',
        'silence']

### 0.4 Initialize pygame mixer

In [26]:
pygame.mixer.init()
print ("The audio modul is up and running.")

The audio modul is up and running.


### 0.5 Define functions

#### 0.5.1 Sound loader

In [27]:
def sound_loader(tag):
    """
    Load all .wav sounds from the specified folder based on the tag.
    
    Parameters:
    tag (str): The tag for the sounds, which is also the folder name (e.g., dog, pig, human).

    Returns:
    list: A list of preloaded pygame Sound objects.
    """
    folder = f'../data_audio/{tag}'
    sounds_list = []
    
    # Load all .wav files from the specified folder
    for file in os.listdir(folder):
        if file.endswith('.wav'):
            sounds_list.append(os.path.join(folder, file))
    
    # Preload sound files
    preloaded_sounds = [pygame.mixer.Sound(file) for file in sounds_list]
    
    print(f"All {tag} sounds preloaded")
    
    return preloaded_sounds

#### 0.5.2 Pseudorandomizer

In [28]:
def pseudorandomizer(tags = TAGS, num_reps = NUM_REPS):
    '''
    In this context "pseudorandom" simply means random shuffle with no
    two subsequent elements being the same.  
    '''
    total_count = num_reps * len(tags)
    tag_counter = Counter(tags * num_reps)
    
    result = []
    
    def add_tag():
        if len(result) == total_count:
            return True
        
        available_tags = [tag for tag in tags if tag_counter[tag] > 0]
        random.shuffle(available_tags)
        
        for tag in available_tags:
            if len(result) == 0 or result[-1] != tag:
                result.append(tag)
                tag_counter[tag] -= 1
                
                if add_tag():
                    return True
                
                # Backtrack
                result.pop()
                tag_counter[tag] += 1
        
        return False
    
    add_tag()
    return result

#### 0.5.3 Sound player

In [29]:
def play_sound(sound):
    '''
    This function is responsible for playing the sound snippets one
    after the other. 
    '''
    sound.play()
    time.sleep(sound.get_length()) 

#### 0.5.4 Send signal to the LUMO computer

In [30]:
def send_signal(message):
    ser.write(message.encode())

### 0.6 Set up the Serial Port

In [31]:
import serial.tools.list_ports

# List all available ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM3 - Intel(R) Active Management Technology - SOL (COM3)


In [32]:
import serial.tools.list_ports

# List all available ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM3 - Intel(R) Active Management Technology - SOL (COM3)


In [33]:
SERIAL_PORT = "COM3"    # Same as on the LUMO device
BAUD_RATE = 9600        # No clue what this is. The manual requests it and ChatGPT suggested this number.
ser = serial.Serial(SERIAL_PORT, BAUD_RATE)

## 1. Preload the sounds
This step is crucial to avoid lagging due to buffering

In [34]:
# Preload all the sounds
for tag in TAGS:
    preloaded_sounds = sound_loader(tag)
    globals()[f"{tag}_preloaded_sounds"] = preloaded_sounds
    print(f"{tag}_preloaded_sounds: {preloaded_sounds}")

All dog sounds preloaded
dog_preloaded_sounds: [<pygame.mixer.Sound object at 0x0000026A870E9020>, <pygame.mixer.Sound object at 0x0000026A870E8390>, <pygame.mixer.Sound object at 0x0000026A870E8210>, <pygame.mixer.Sound object at 0x0000026A870E87E0>, <pygame.mixer.Sound object at 0x0000026A870E8D20>, <pygame.mixer.Sound object at 0x0000026A870E8300>, <pygame.mixer.Sound object at 0x0000026A870E8930>, <pygame.mixer.Sound object at 0x0000026A870E9A40>, <pygame.mixer.Sound object at 0x0000026A870E8A50>, <pygame.mixer.Sound object at 0x0000026A870E9AD0>, <pygame.mixer.Sound object at 0x0000026A870E8D50>, <pygame.mixer.Sound object at 0x0000026A870E9A10>, <pygame.mixer.Sound object at 0x0000026A870E8DE0>, <pygame.mixer.Sound object at 0x0000026A870E8540>, <pygame.mixer.Sound object at 0x0000026A870E8F90>, <pygame.mixer.Sound object at 0x0000026A870E8810>, <pygame.mixer.Sound object at 0x0000026A870E88D0>, <pygame.mixer.Sound object at 0x0000026A870E86F0>, <pygame.mixer.Sound object at 0x00

## 2. Create the pseudorandom sequence of blocks
A more proper term could be non-repetitive random or constrained random. But I just stick with pseudorandom. The idea is that the blocks are randomized but two consecutive blocks cannot be the same. 

In [35]:
block_sequence = pseudorandomizer()
print(len(block_sequence), block_sequence)

15 ['swine', 'human', 'swine', 'environment', 'silence', 'human', 'environment', 'silence', 'swine', 'human', 'dog', 'silence', 'dog', 'environment', 'dog']


## 3. Run the experiment

Each block will be 6 seconds long. (Snippets are 0.5 seconds, so 12 of them are needed for one block.) The snippets are selected randomly from the audio pool. There is also a silence block. That's simply generated as 6 seconds of silence.v

In [36]:
# Function to play a sound file and wait for it to finish
print("Experiment starts now.")
for block in block_sequence:
    send_signal(f"start {block}")
    preloaded_sounds = eval(f"{block}_preloaded_sounds")
    play_this = random.sample(preloaded_sounds, BLOCK_LENGTH*2)
    for snippet in play_this:
        play_sound(snippet)
    print(f"{block} sounds played")
    send_signal(f"end {block}")
    time.sleep(2)
    print("2 sec pause over")
print("Experiment is over.")

Experiment starts now.
swine sounds played
2 sec pause over
human sounds played
2 sec pause over
swine sounds played
2 sec pause over
environment sounds played
2 sec pause over
silence sounds played
2 sec pause over
human sounds played
2 sec pause over
environment sounds played
2 sec pause over
silence sounds played
2 sec pause over
swine sounds played
2 sec pause over
human sounds played
2 sec pause over
dog sounds played
2 sec pause over
silence sounds played
2 sec pause over
dog sounds played
2 sec pause over
environment sounds played
2 sec pause over
dog sounds played
2 sec pause over
Experiment is over.
